In [1]:
import cv2
import numpy as np
import pywt
import matplotlib.pyplot as plt
import math

In [2]:
image = cv2.imread('/home/juanca632/PersonalProjects/stage/wavelet-transform/image_gray.jpg', cv2.IMREAD_GRAYSCALE)

In [3]:
# Crear una matriz 10x10 con valores aleatorios enteros entre 0 y 255
matrix = np.random.randint(0, 256, size=(10, 10))

# Imprimir la matriz
print(matrix)

[[234  86 105 126 129 137   6 229 110 121]
 [251 118  54 211 224 244 171  35 219  17]
 [102 157  85  38 238  45 221 238 120 168]
 [161  79 160  18 104  55 181 163 123 255]
 [ 72  80 124  18  10 134  60 212 165   5]
 [167 194  60 124 175 226 145 191 180  47]
 [255  27  77  70  89  80  36  79  58 144]
 [237 211  38 137  69  31  94  31 254 116]
 [176 116  27 198  78  42  46 137 254 240]
 [144 199 117 120 236 211  15 118  12  64]]


In [ ]:
def haar_wavelet_transform(matrix):
    n = int(matrix.shape[0])
    n_half = int(n /2)
    transformed_matrix = np.zeros((n,n))

    if(n >= 2):
        for i in range(0, n):
            aux = 0
            aux2 = int(n_half + aux)
            for j in range(0, n, 2):

                sum = (matrix[i, j] + matrix[i, j+1]) / 2
                dif = (matrix[i, j] - matrix[i, j+1]) / 2
                transformed_matrix[i,aux] = sum
                transformed_matrix[i,aux2] = dif
                aux = aux + 1
                aux2 = aux2 + 1
        transformed_matrix2 = np.copy(transformed_matrix)
        for j in range(0, n_half):
            aux = 0
            aux2 = int(n_half + aux)
            for i in range(0, n, 2):

                sum = (transformed_matrix[i, j] + transformed_matrix[i+1, j]) / 2
                dif = (transformed_matrix[i, j] - transformed_matrix[i+1, j]) / 2

                transformed_matrix2[aux,j] = sum
                transformed_matrix2[aux2,j] = dif
                aux = aux + 1
                aux2 = aux2 + 1

        transformed_matrix3 = transformed_matrix2[:n_half,:n_half]

    return transformed_matrix2, transformed_matrix3

image_result, image_aprox = haar_wavelet_transform(matrix)
    
print("\n")
# print(image_result)
plt.imshow(image_aprox,  cmap="gray")
plt.show()

